In [4]:
import cv2
import mediapipe as mp
import time

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.8,
    min_tracking_confidence=0.8
)
mp_draw = mp.solutions.drawing_utils

# Volume and control variables
volume_level = 50  #
on_off_state = False  
pinched = False
last_toggle_time = 0

# Swipe tracking
previous_index_y = None
volume_change_delay = 0.1
last_volume_change_time = 0

# Pinch detection
PINCH_TOGGLE_THRESHOLD = 0.03

# Hand lost detection
hand_detected = False
last_hand_time = 0
HAND_TIMEOUT = 1.5  # seconds

# Finger detection

def detection(hand_landmarks, hand_label):
    fingers = []
    landmarks = hand_landmarks.landmark
    tip_ids = [4, 8, 12, 16, 20]

    # Thumb detection based on hand side
    if hand_label == "Right":
        fingers.append(1 if landmarks[tip_ids[0]].x < landmarks[tip_ids[0] - 1].x else 0)
    else:
        fingers.append(1 if landmarks[tip_ids[0]].x > landmarks[tip_ids[0] - 1].x else 0)

    # Other fingers
    for id in range(1, 5):
        fingers.append(1 if landmarks[tip_ids[id]].y < landmarks[tip_ids[id] - 2].y else 0)

    return fingers

# calculating distance

def distance_between_points(p1, p2):
    return ((p1.x - p2.x)**2 + (p1.y - p2.y)**2) ** 0.5

# Webcam

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)   # Width
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)   # Height

cv2.namedWindow("Hand Tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Hand Tracking", 1280, 720)


while True:
    ret, frame = capture.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # mirror view
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    current_time = time.time()

    if results.multi_hand_landmarks:
        hand_detected = True
        last_hand_time = current_time

        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            hand_label = results.multi_handedness[idx].classification[0].label
            landmarks = hand_landmarks.landmark

            # Get bounding box
            h, w, _ = frame.shape
            x_list = [int(lm.x * w) for lm in landmarks]
            y_list = [int(lm.y * h) for lm in landmarks]
            x_min, x_max = min(x_list), max(x_list)
            y_min, y_max = min(y_list), max(y_list)

            cv2.rectangle(frame, (x_min - 20, y_min - 20), (x_max + 20, y_max + 20), (255, 0, 0), 2)

            # Finger detection
            finger_states = detection(hand_landmarks, hand_label)
            total_fingers = sum(finger_states)

            # on / off
            thumb_tip = landmarks[4]
            index_tip = landmarks[8]
            pinch_distance = distance_between_points(thumb_tip, index_tip)

            middle_finger = finger_states[2]
            ring_finger = finger_states[3]
            pinky_finger = finger_states[4]

            if pinch_distance < 0.03 and finger_states[2] == 1 and finger_states[3] == 1 and finger_states[4] == 1:
                 gesture = "OK Sign"
            

            # Gesture name (for display)
            if total_fingers == 0:
                gesture = "Fist"
            elif total_fingers == 5:
                gesture = "Open Palm"
            elif finger_states == [0, 1, 1, 0, 0]:
                gesture = "Peace Sign"
            elif finger_states == [1, 0, 0, 0, 0]:
                gesture = "Thumbs Up"
            elif finger_states == [1, 1, 0, 0, 1]:
                gesture = "Watch dogs : You're being watched "
            else:
                gesture = f"{total_fingers} fingers up"
                

            display_text = f"Fingers: {total_fingers} | Gesture: {gesture}"

            
            if pinch_distance < PINCH_TOGGLE_THRESHOLD:
                if not pinched and current_time - last_toggle_time > 1.0:
                    on_off_state = not on_off_state
                    last_toggle_time = current_time
                pinched = True
            else:
                pinched = False

            # Vol control

            if on_off_state and finger_states == [0, 1, 0, 0, 0]:  # only index up
                index_y = landmarks[8].y
                if previous_index_y is not None:
                    delta_y = previous_index_y - index_y
                    if abs(delta_y) > 0.007 and current_time - last_volume_change_time > volume_change_delay:
                        if delta_y > 0:
                            volume_level = min(100, volume_level + 5)  # swipe up
                        else:
                            volume_level = max(0, volume_level - 5)  # swipe down
                        last_volume_change_time = current_time
                previous_index_y = index_y
            else:
                previous_index_y = None  # reset when not in swipe gesture

            
            # display

            cv2.putText(frame, f"{hand_label} Hand", (x_min - 20, y_min - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

            cv2.putText(frame, display_text, (x_min - 20, y_max + 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            cv2.putText(frame, f"System: {'ON' if on_off_state else 'OFF'}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

            cv2.putText(frame, f"Volume: {int(volume_level)}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

    else:
        # not detected
        if current_time - last_hand_time > HAND_TIMEOUT:
            hand_detected = False

    if not hand_detected:
        cv2.putText(frame, "Hand Lost", (10, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)

    # Show full screen
    cv2.imshow("Hand Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
capture.release()
cv2.destroyAllWindows()
